Authors:

Kenneth Yam ID: 61791166

Rodrigo Erquiaga ID:  43484513

David Gathright ID: 15478532

# CS 179 Final Report: Using TrueSkill to predict DotA tournament winnings

Code corresponding to the Final Report for Spring Quarter CS 179 Final Project.

### Imports and TrueSkill set up

In [1]:
import trueskill as ts
from trueskill.backends import cdf
import json
import urllib2
from math import sqrt
ts.TrueSkill(draw_probability=0, backend='scipy')

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%, backend='scipy')

### Creating Players and Teams

In [2]:
# Team Indexes
team_names = [
    'Virtus.Pro', 
    'Team Liquid', 
    'PSG.LGD', 
    'Team Secret', 
    'Mineski', 
    'Vici Gaming',
    'Newbee',
    'VGJ.Thunder',
    'OG', 
    'Winstrike Team',
    'Team Serenity',
    'Invictus Gaming2',
    'Fnatic',
    'TNC Predator',
    'VGJ.Storm',
    'Evil Geniuses',
    'OpTic Gaming',
    'paiN Gaming',
    'Natus Vincere',
    'FlyToMoon',
    'LGD.Forever Young',
    'Complexity Gaming',
    'Going In',
    'Immortals',
    'Team Kinguin',
    'Vega Squadron',
    'Keen Gaming',
    'Echo International',
    'SG e-sports',
    'Digital Chaos',
    'The Final Tribe',
    'Rex Regum QEON',
    'Infamous',
    'Team Empire',
    'Invictus Gaming',
    'LGD Gaming',
    'iG Vitality',
    'Team Spirit',
]

team_index = {} # Used to find team based by name

for index, name in enumerate(team_names):
    team_index[name] = index

# Creating teams
teams = [[ts.Rating() for _ in range(5)] for _ in range(len(team_names))]

# Example use
# print "Newbee Team before win: "
# for index, player in enumerate(teams[team_index['Newbee']]):
#     print "Player {}: ".format(index+1), player

# teams[team_index['Newbee']], teams[team_index['VGJ.Thunder']] = trueskill.rate([teams[team_index['Newbee']], teams[team_index['VGJ.Thunder']]], ranks=[0, 1])

# print "Newbee Team after win: "
# for index, player in enumerate(teams[team_index['Newbee']]):
#     print "Player {}: ".format(index+1), player

### Learning from Training Data

We got the training data (win/loss between team matches in playoffs) from the Dota Pro Circuit 2017-2018 games. Each section contains the name of the tournament as a comment with the match data in chronological order in the list `matches`. Then we traversed the matches to update the TrueSkill value of our teams.

In [3]:
matches = [] # [winner, loser, win_count, lose_count] # win_count is number of wins for winner, lose_count is number of wins for loser

# StarLadder i-League Invitational Season 3
matches.append( [ 'Complexity Gaming', 'Mineski', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Team Secret', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Mineski', 3, 1 ] )

# PGL Open Bucharest
matches.append( [ 'Mineski', 'Immortals', 2, 1 ] )
matches.append( [ 'PSG.LGD', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Mineski', 'PSG.LGD', 2, 0 ] )

# ESL One Hamburg 2017
matches.append( [ 'Team Secret', 'Team Liquid', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Newbee', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Team Secret', 2, 0 ] )

# AMD Sapphire Dota PIT League
matches.append( [ 'Newbee', 'SG e-sports', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Immortals', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'OG', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Fnatic', 2, 1 ] )
matches.append( [ 'SG e-sports', 'Immortals', 1, 0 ] )
matches.append( [ 'OG', 'Fnatic', 1, 0 ] )
matches.append( [ 'Team Liquid', 'Newbee', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Virtus.Pro', 'SG e-sports', 1, 0 ] )
matches.append( [ 'Newbee', 'OG', 1, 0 ] )
matches.append( [ 'Newbee', 'Virtus.Pro', 1, 0 ] )
matches.append( [ 'Team Liquid', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Newbee', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Vici Gaming', 3, 2 ] )

# Perfect World Masters
matches.append( [ 'Team Secret', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Mineski', 2, 0 ] )
matches.append( [ 'Newbee', 'LGD.Forever Young', 2, 1 ] )
matches.append( [ 'Team Kinguin', 'Vega Squadron', 2, 1 ] )
matches.append( [ 'Complexity Gaming', 'Mineski', 1, 0 ] )
matches.append( [ 'LGD.Forever Young', 'Vega Squadron', 1, 0 ] )
matches.append( [ 'Vici Gaming', 'Team Secret', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Kinguin', 2, 0 ] )
matches.append( [ 'Complexity Gaming', 'Team Kinguin', 2, 1 ] )
matches.append( [ 'LGD.Forever Young', 'Team Secret', 2, 0 ] )
matches.append( [ 'LGD.Forever Young', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Newbee', 'Vici Gaming', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'LGD.Forever Young', 2, 0 ] )
matches.append( [ 'Newbee', 'Vici Gaming', 3, 0 ] )

# DreamLeague Season 8
matches.append( [ 'Virtus.Pro', 'Natus Vincere', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Fnatic', 2, 0 ] )
matches.append( [ 'Team Secret', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Newbee', 'Infamous', 2, 0 ] )
matches.append( [ 'Natus Vincere', 'Fnatic', 2, 1 ] )
matches.append( [ 'Evil Geniuses', 'Infamous', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Virtus.Pro', 2, 1 ] )
matches.append( [ 'Team Secret', 'Newbee', 2, 0 ] )
matches.append( [ 'Natus Vincere', 'Newbee', 2, 1 ] )
matches.append( [ 'Evil Geniuses', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Evil Geniuses', 'Natus Vincere', 2, 0 ] )
matches.append( [ 'Team Secret', 'Team Liquid', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Team Secret', 'Team Liquid', 3, 0 ] )

# MDL Macau
matches.append( [ 'OG', 'Natus Vincere', 2, 0 ] )
matches.append( [ 'TNC Predator', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'OG', 'TNC Predator', 2, 0 ] )

# DOTA Summit 8
matches.append( [ 'Complexity Gaming', 'OpTic Gaming', 2, 1 ] )
matches.append( [ 'OG', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'OG', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Team Kinguin', 2, 0 ] )
matches.append( [ 'Fnatic', 'OG', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Fnatic', 3, 1 ] )

# Captains Draft 4.0
matches.append( [ 'Mineski', 'paiN Gaming', 2, 0 ] )
matches.append( [ 'Team Secret', 'Evil Geniuses', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Complexity Gaming', 2, 0 ] )
matches.append( [ 'OG', 'Team Empire', 2, 0 ] )
matches.append( [ 'Team Secret', 'Mineski', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'OG', 2, 0 ] )
matches.append( [ 'Team Secret', 'Vici Gaming', 3, 2 ] )

# ESL One Genting 2018
matches.append( [ 'Newbee', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'Evil Geniuses', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Secret', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Liquid', 3, 2 ] )

# StarLadder i-League Invitational Season 4
matches.append( [ 'PSG.LGD', 'Newbee', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Mineski', 2, 0 ] )
matches.append( [ 'Team Liquid', 'PSG.LGD', 3, 1 ] )

# ESL One Katowice
matches.append(['Team Liquid', 'Team Secret', 2, 1])
matches.append(['Virtus.Pro', 'Evil Geniuses', 2, 1])
matches.append(['Vici Gaming', 'Team Liquid', 2, 1])
matches.append(['Virtus.Pro', 'Fnatic', 2, 0])
matches.append(['Virtus.Pro', 'Vici Gaming', 3, 1])
matches.append(['Team Liquid', 'Fnatic', 2, 1])

# Bucharest Major
matches.append(['VGJ.Thunder', 'TNC Predator', 2, 1])
matches.append(['Newbee', 'Team Secret', 2, 0])
matches.append(['Team Liquid', 'Evil Geniuses', 2, 1])
matches.append(['Virtus.Pro', 'OpTic Gaming', 2, 0])
matches.append(['VGJ.Thunder', 'Newbee', 2, 0])
matches.append(['Virtus.Pro', 'Team Liquid', 2, 1])
matches.append(['Virtus.Pro', 'VGJ.Thunder', 3, 0])

# Indonesia Minor
matches.append(['Natus Vincere', 'Digital Chaos', 1, 0])
matches.append(['The Final Tribe', 'Rex Regum QEON', 1, 0])
matches.append(['Natus Vincere', 'Fnatic', 2, 1])
matches.append(['Infamous', 'The Final Tribe', 2, 1])
matches.append(['Evil Geniuses', 'Natus Vincere', 2, 0])
matches.append(['VGJ.Thunder', 'Infamous', 2, 1])
matches.append(['Evil Geniuses', 'VGJ.Thunder', 2, 0])

# DreamLeague Season 9
matches.append(['Team Liquid', 'paiN Gaming', 2, 0])
matches.append(['Fnatic', 'OG', 2, 0])
matches.append(['Newbee', 'Immortals', 2, 0])
matches.append(['Team Secret', 'Team Empire', 2, 0])
matches.append(['Team Liquid', 'Fnatic', 2, 1])
matches.append(['Team Secret', 'Newbee', 2, 0])
matches.append(['OG', 'paiN Gaming', 2, 0])
matches.append(['Team Empire', 'Immortals', 2, 1])
matches.append(['Newbee', 'OG', 2, 0])
matches.append(['Fnatic', 'Team Empire', 2, 0])
matches.append(['Fnatic', 'Newbee', 2, 1])
matches.append(['Team Secret', 'Team Liquid', 2, 0])
matches.append(['Fnatic', 'Team Liquid', 2, 0])
matches.append(['Team Secret', 'Fnatic', 3, 0])

# DotA 2 Asia Championships
matches.append(['Evil Geniuses', 'Newbee', 2, 0])
matches.append(['Team Liquid', 'VGJ.Thunder', 2, 0])
matches.append(['OpTic Gaming', 'Invictus Gaming', 2, 0])
matches.append(['Vici Gaming', 'Team Secret', 2, 0])
matches.append(['LGD Gaming', 'Evil Geniuses', 2, 0])
matches.append(['Team Liquid', 'Virtus.Pro', 2, 0])
matches.append(['Mineski', 'OpTic Gaming', 2, 0])
matches.append(['Vici Gaming', 'TNC Predator', 2, 1])
matches.append(['Virtus.Pro', 'Evil Geniuses', 1, 0])
matches.append(['TNC Predator', 'OpTic Gaming', 1, 0])
matches.append(['LGD Gaming', 'Team Liquid', 2, 1])
matches.append(['Mineski', 'Vici Gaming', 2, 1])
matches.append(['Virtus.Pro', 'Vici Gaming', 1, 0])
matches.append(['TNC Predator', 'Team Liquid', 1, 0])
matches.append(['Virtus.Pro', 'TNC Predator', 2, 1])
matches.append(['Mineski', 'LGD Gaming', 2, 0])
matches.append(['LGD Gaming', 'Virtus.Pro', 2, 1])
matches.append(['Mineski', 'LGD Gaming', 3, 2])

# StarLadder ImbaTV Invitational Season 5
matches.append(['VGJ.Thunder', 'Vega Squadron', 2, 0])
matches.append(['OpTic Gaming', 'Fnatic', 2, 1])
matches.append(['OpTic Gaming', 'VGJ.Thunder', 3, 1])

# EPICENTER XL
matches.append(['FlyToMoon', 'paiN Gaming', 1, 0])
matches.append(['Mineski', 'Team Secret', 1, 0])
matches.append(['Team Liquid', 'OG', 2, 0])
matches.append(['PSG.LGD', 'Virtus.Pro', 2, 1])
matches.append(['FlyToMoon', 'OG', 1, 0])
matches.append(['Virtus.Pro', 'Mineski', 1, 0])
matches.append(['FlyToMoon', 'Virtus.Pro', 2, 0])
matches.append(['PSG.LGD', 'Team Liquid', 2, 0])
matches.append(['Team Liquid', 'FlyToMoon', 2, 1])
matches.append(['PSG.LGD', 'Team Liquid', 3, 1])

# Thailand Minor
matches.append(['Keen Gaming', 'Evil Geniuses', 2, 1])
matches.append(['VGJ.Storm', 'The Final Tribe', 2, 1])
matches.append(['Keen Gaming', 'Team Secret', 2, 1])
matches.append(['VGJ.Storm', 'Fnatic', 2, 0])
matches.append(['VGJ.Storm', 'Keen Gaming', 2, 0])

# MDL Changsha Major
matches.append( [ 'Newbee', 'Invictus Gaming', 1, 0 ] )
matches.append( [ 'PSG.LGD', 'iG Vitality', 1, 0 ] )
matches.append( [ 'VGJ.Storm', 'OG', 2, 0 ] )
matches.append( [ 'Mineski', 'TNC Predator', 2, 0 ] )
matches.append( [ 'Newbee', 'OG', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'TNC Predator', 2, 1 ] )
matches.append( [ 'VGJ.Storm', 'Team Secret', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Mineski', 2, 1 ] )
matches.append( [ 'Newbee', 'Mineski', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'Team Secret', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'Newbee', 2, 0 ] )
matches.append( [ 'VGJ.Storm', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'VGJ.Storm', 3, 0 ] )

# ESL One Birmingham 2018
matches.append( [ 'paiN Gaming', 'Mineski', 2, 0 ] )
matches.append( [ 'Fnatic', 'OG', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'paiN Gaming', 2, 0 ] )
matches.append( [ 'OpTic Gaming', 'Fnatic', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'OpTic Gaming', 3, 0 ] )
matches.append( [ 'paiN Gaming', 'Fnatic', 2, 1 ] )

# China Dota2 Supermajor
matches.append( [ 'Evil Geniuses', 'Team Spirit', 2, 1 ] )
matches.append( [ 'VGJ.Storm', 'Infamous', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Natus Vincere', 2, 1 ] )
matches.append( [ 'VGJ.Thunder', 'The Final Tribe', 2, 0 ] )
matches.append( [ 'Team Liquid', 'TNC Predator', 2, 0 ] )
matches.append( [ 'Team Secret', 'OpTic Gaming', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Newbee', 2, 1 ] )
matches.append( [ 'PSG.LGD', 'Mineski', 2, 1 ] )
matches.append( [ 'OpTic Gaming', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'TNC Predator', 'VGJ.Storm', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Mineski', 2, 1 ] )
matches.append( [ 'Newbee', 'VGJ.Thunder', 2, 0 ] )
matches.append( [ 'OpTic Gaming', 'TNC Predator', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Newbee', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Team Secret', 2, 0 ] )
matches.append( [ 'PSG.LGD', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Virtus.Pro', 'OpTic Gaming', 2, 0 ] )
matches.append( [ 'Team Secret', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'Virtus.Pro', 'Team Secret', 2, 1 ] )
matches.append( [ 'Team Liquid', 'PSG.LGD', 2, 0 ] )
matches.append( [ 'Virtus.Pro', 'PSG.LGD', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Virtus.Pro', 3, 2 ] )


total_number_of_matches = 0
for winner, loser, win_count, lose_count in matches:
    for _ in range(win_count):
        teams[team_index[winner]], teams[team_index[loser]] = ts.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[0, 1])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])
        total_number_of_matches += 1
        
    for _ in range(lose_count):
        teams[team_index[winner]], teams[team_index[loser]] = ts.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[1, 0])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])
        total_number_of_matches += 1
print "Total Matches: ", total_number_of_matches

Total Matches:  450


In [4]:
teams

[[trueskill.Rating(mu=29.978, sigma=2.603),
  trueskill.Rating(mu=29.978, sigma=2.603),
  trueskill.Rating(mu=29.978, sigma=2.603),
  trueskill.Rating(mu=29.978, sigma=2.603),
  trueskill.Rating(mu=29.978, sigma=2.603)],
 [trueskill.Rating(mu=30.271, sigma=2.535),
  trueskill.Rating(mu=30.271, sigma=2.535),
  trueskill.Rating(mu=30.271, sigma=2.535),
  trueskill.Rating(mu=30.271, sigma=2.535),
  trueskill.Rating(mu=30.271, sigma=2.535)],
 [trueskill.Rating(mu=29.426, sigma=3.628),
  trueskill.Rating(mu=29.426, sigma=3.628),
  trueskill.Rating(mu=29.426, sigma=3.628),
  trueskill.Rating(mu=29.426, sigma=3.628),
  trueskill.Rating(mu=29.426, sigma=3.628)],
 [trueskill.Rating(mu=28.186, sigma=2.906),
  trueskill.Rating(mu=28.186, sigma=2.906),
  trueskill.Rating(mu=28.186, sigma=2.906),
  trueskill.Rating(mu=28.186, sigma=2.906),
  trueskill.Rating(mu=28.186, sigma=2.906)],
 [trueskill.Rating(mu=26.781, sigma=3.463),
  trueskill.Rating(mu=26.781, sigma=3.463),
  trueskill.Rating(mu=26.781

### Function to Compare WInning Chance between Two Players

In [5]:
def win_chance(p1, p2): #chance p1 beats p2
    deltaMu = p1.mu - p2.mu
    rsss = sqrt(p1.sigma**2 + p2.sigma**2)
    return cdf(deltaMu/rsss)

def get_winner(nameA, nameB):
    teamA = teams[team_index[nameA]]
    teamB = teams[team_index[nameB]]
    if win_chance(teamA[0], teamB[0]):   # use the first player instead of sum of all because each player has same TrueSkill score.
        return nameA                     # refer to Future Enhancement below 
    else:
        return nameB

def update_winner(nameA, nameB):
    teamA = teams[team_index[nameA]]
    teamB = teams[team_index[nameB]]
    if win_chance(teamA[0], teamB[0]):
        winner = nameA
        loser = nameB
        teams[team_index[winner]], teams[team_index[loser]] = ts.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[0, 1])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])  
        return nameA
    else:
        winner = nameB
        loser = nameA
        teams[team_index[winner]], teams[team_index[loser]] = ts.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[0, 1])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])  
        return nameB


### Predicting our Test Playoff: The International 2018

In [6]:
## Tournament: The International

predict_matches_upper = [] # [winner, loser] # win_count is number of wins for winner, lose_count is number of wins for loser
predict_matches_lower = []

predict_matches_upper.append( [ 'Team Liquid', 'OpTic Gaming'] )
predict_matches_upper.append( [ 'Virtus.Pro', 'PSG.LGD'] )
predict_matches_upper.append( [ 'VGJ.Storm', 'OG'] )
predict_matches_upper.append( [ 'Evil Geniuses', 'Team Secret' ] )
predict_matches_lower.append( [ 'Team Serenity', 'Fnatic'] )
predict_matches_lower.append( [ 'TNC Predator', 'Mineski' ] )
predict_matches_lower.append( [ 'Newbee', 'Winstrike Team' ] )
predict_matches_lower.append( [ 'VGJ.Thunder', 'Vici Gaming' ] )

new_predict_matches_upper = []
new_predict_matches_lower = []

for nameA, nameB in predict_matches_lower:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    new_predict_matches_lower.append([winner])

lower_index = 0
upper_list = []
for nameA, nameB in predict_matches_upper:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    upper_list.append(winner)
    if(winner == nameA):
        new_predict_matches_lower[lower_index].append(nameB)
    else:
        new_predict_matches_lower[lower_index].append(nameA)
    lower_index+=1
    
for i in range(0, len(upper_list), 2):
    new_predict_matches_upper.append([upper_list[i], upper_list[i+1]])

print new_predict_matches_upper
print new_predict_matches_lower

print '----------'

predict_matches_upper = new_predict_matches_upper 
predict_matches_lower = new_predict_matches_lower

new_predict_matches_upper = []
new_predict_matches_lower = []

lower_list = []
for nameA, nameB in predict_matches_lower:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    lower_list.append(winner)

for i in range(0, len(lower_list), 2):
    new_predict_matches_lower.append([lower_list[i], lower_list[i+1]])


upper_list = []
temp_lower = []
for nameA, nameB in predict_matches_upper:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    upper_list.append(winner)
    if(winner == nameA):
        temp_lower.append(nameB)
    else:
        temp_lower.append(nameA)

for i in range(0, len(upper_list), 2):
    new_predict_matches_upper.append([upper_list[i], upper_list[i+1]])

print new_predict_matches_upper
print new_predict_matches_lower


print '----------'

predict_matches_upper = new_predict_matches_upper 
predict_matches_lower = new_predict_matches_lower

new_predict_matches_upper = []
new_predict_matches_lower = []

for nameA, nameB in predict_matches_lower:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    new_predict_matches_lower.append([winner])


upper_list = []
for nameA, nameB in predict_matches_upper:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    upper_list.append(winner)

for index in range(len(temp_lower)):
    new_predict_matches_lower[index].append(temp_lower[index])

# get winner from upper
upper_winner = update_winner(predict_matches_upper[0][0], predict_matches_upper[0][1])


print new_predict_matches_upper
print new_predict_matches_lower

print '----------'

predict_matches_upper = new_predict_matches_upper 
predict_matches_lower = new_predict_matches_lower

new_predict_matches_lower = []
# get winner from lower
lower_list = []
for nameA, nameB in predict_matches_lower:
    winner = update_winner(nameA, nameB)
    print "Winner: ", winner
    lower_list.append(winner)

for i in range(0, len(lower_list), 2):
    new_predict_matches_lower.append([lower_list[i], lower_list[i+1]])

print new_predict_matches_upper
print new_predict_matches_lower

print '----------'

predict_matches_upper = new_predict_matches_upper 
predict_matches_lower = new_predict_matches_lower

lower_winner = update_winner(predict_matches_lower[0][0], predict_matches_lower[0][1])

# get final winner
final_winner = update_winner(predict_matches_lower[0][0], predict_matches_lower[0][1])
print "Final Winner: ", final_winner

Winner:  Team Serenity
Winner:  TNC Predator
Winner:  Newbee
Winner:  VGJ.Thunder
Winner:  Team Liquid
Winner:  Virtus.Pro
Winner:  VGJ.Storm
Winner:  Evil Geniuses
[['Team Liquid', 'Virtus.Pro'], ['VGJ.Storm', 'Evil Geniuses']]
[['Team Serenity', 'OpTic Gaming'], ['TNC Predator', 'PSG.LGD'], ['Newbee', 'OG'], ['VGJ.Thunder', 'Team Secret']]
----------
Winner:  Team Serenity
Winner:  TNC Predator
Winner:  Newbee
Winner:  VGJ.Thunder
Winner:  Team Liquid
Winner:  VGJ.Storm
[['Team Liquid', 'VGJ.Storm']]
[['Team Serenity', 'TNC Predator'], ['Newbee', 'VGJ.Thunder']]
----------
Winner:  Team Serenity
Winner:  Newbee
Winner:  Team Liquid
[]
[['Team Serenity', 'Virtus.Pro'], ['Newbee', 'Evil Geniuses']]
----------
Winner:  Team Serenity
Winner:  Newbee
[]
[['Team Serenity', 'Newbee']]
----------
Final Winner:  Team Serenity


In [7]:
teams

[[trueskill.Rating(mu=29.727, sigma=2.553),
  trueskill.Rating(mu=29.727, sigma=2.553),
  trueskill.Rating(mu=29.727, sigma=2.553),
  trueskill.Rating(mu=29.727, sigma=2.553),
  trueskill.Rating(mu=29.727, sigma=2.553)],
 [trueskill.Rating(mu=31.134, sigma=2.481),
  trueskill.Rating(mu=31.134, sigma=2.481),
  trueskill.Rating(mu=31.134, sigma=2.481),
  trueskill.Rating(mu=31.134, sigma=2.481),
  trueskill.Rating(mu=31.134, sigma=2.481)],
 [trueskill.Rating(mu=27.998, sigma=3.524),
  trueskill.Rating(mu=27.998, sigma=3.524),
  trueskill.Rating(mu=27.998, sigma=3.524),
  trueskill.Rating(mu=27.998, sigma=3.524),
  trueskill.Rating(mu=27.998, sigma=3.524)],
 [trueskill.Rating(mu=26.952, sigma=2.848),
  trueskill.Rating(mu=26.952, sigma=2.848),
  trueskill.Rating(mu=26.952, sigma=2.848),
  trueskill.Rating(mu=26.952, sigma=2.848),
  trueskill.Rating(mu=26.952, sigma=2.848)],
 [trueskill.Rating(mu=26.133, sigma=3.420),
  trueskill.Rating(mu=26.133, sigma=3.420),
  trueskill.Rating(mu=26.133

### Future Enhancement: Collecting Pro Player ID from OpenDotA API

Another factor to consider is that our TrueSkill calculation was limited just between the teams and failed to take into account different rankings for different players in each team. One tweak to make our calculations more realistic would be to take into account data from individual Pro Players before assembling the teams, as well as in between tournamnets. We can use OpenDotA API to gain access to some data on some of these pro players as follows:

In [ ]:
url = "https://api.opendota.com/api/proPlayers"
header = {'User-Agent': 'Mozilla/5.0'}
request = urllib2.Request(url,headers=header)
data = json.load(urllib2.urlopen(request))
# print data

In [ ]:
pro_names = {}
for index, account in enumerate(data):
    if account['team_name'] in team_names:
        pro_names[account['name']] = index
        print "Team Name:", account['team_name'], ",\tPlayer: ", account['name']
    
# data_teams = set()
# for index, account in enumerate(data):
#     data_teams.add(account['team_name'])
#     if account['team_name'] == 'Newbee':
#         print(account['name'])
# print data_teams
